# Note: Modify the paths to the relevant data.

# 1. Import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import gensim
import gensim.corpora as corpora

In [ ]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# 2. Read_csv

In [ ]:
post=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/POST_cleaned.csv')
comment=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/COMMENT_cleaned.csv')
user=pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Cleaned/USER_cleaned.csv')
test= pd.read_csv('/content/drive/MyDrive/DS300/RS_DA/DATA_vfo/DATASET/Test/test.csv')

In [ ]:
drop_u=test[test['id_post']==4008]['id_user'].values
test=test.drop(list(test[test['id_user'].isin(drop_u)].index.values),axis=0)
test

,Unnamed: 0,id_user,id_post,label
0,0,179483,16,1
1,1,179483,21753,0
2,2,179483,15535,0
3,3,179483,15045,0
4,4,179483,3773,0
...,...,...,...,...
50495,50495,386961,20632,0
50496,50496,386961,17499,0
50497,50497,386961,20530,0
50498,50498,386961,1921,0


In [ ]:
user

,id_user,name_user,star,DateJoin,DateLastAccess,comment_count,Reaction_Score,Score
0,179483,0406,1.0,2013-10-24T16:03:00+0700,NaN,23480,99,1132
1,468274,Clarita,1.0,2021-10-29T16:13:29+0700,2021-11-02T09:03:36+0700,1,0,1
2,460618,thien-ha-bet,1.0,2020-07-13T16:17:58+0700,2021-11-01T16:53:54+0700,7,0,1
3,468272,laptoptitan.vn,1.0,2021-10-29T10:29:04+0700,2021-10-29T10:48:30+0700,1,0,1
4,468270,Phúc Lucas,1.0,2021-10-29T09:42:46+0700,2021-10-31T15:20:36+0700,2,0,1
...,...,...,...,...,...,...,...,...
10991,51414,hoangpupu,NaN,2011-07-11T01:40:44+0700,2012-11-21T01:55:12+0700,37,0,0
10992,33680,..::Rùa::..,NaN,2011-05-26T19:21:00+0700,2013-02-27T23:20:36+0700,337,0,0
10993,20,^^KuTi_K_,NaN,2010-08-11T11:30:00+0700,2014-08-24T15:57:53+0700,379,0,0
10994,348,NhocShjnky,NaN,2010-11-20T22:42:19+0700,2010-11-20T22:54:18+0700,1,0,0


In [ ]:
norm_col=['comment_count','Reaction_Score','Score']
user[norm_col]=MinMaxScaler().fit_transform(user[norm_col])
user

,id_user,name_user,star,DateJoin,DateLastAccess,comment_count,Reaction_Score,Score
0,179483,0406,1.0,2013-10-24T16:03:00+0700,NaN,0.878907,0.970588,1.000000
1,468274,Clarita,1.0,2021-10-29T16:13:29+0700,2021-11-02T09:03:36+0700,0.000037,0.000000,0.000883
2,460618,thien-ha-bet,1.0,2020-07-13T16:17:58+0700,2021-11-01T16:53:54+0700,0.000262,0.000000,0.000883
3,468272,laptoptitan.vn,1.0,2021-10-29T10:29:04+0700,2021-10-29T10:48:30+0700,0.000037,0.000000,0.000883
4,468270,Phúc Lucas,1.0,2021-10-29T09:42:46+0700,2021-10-31T15:20:36+0700,0.000075,0.000000,0.000883
...,...,...,...,...,...,...,...,...
10991,51414,hoangpupu,NaN,2011-07-11T01:40:44+0700,2012-11-21T01:55:12+0700,0.001385,0.000000,0.000000
10992,33680,..::Rùa::..,NaN,2011-05-26T19:21:00+0700,2013-02-27T23:20:36+0700,0.012615,0.000000,0.000000
10993,20,^^KuTi_K_,NaN,2010-08-11T11:30:00+0700,2014-08-24T15:57:53+0700,0.014187,0.000000,0.000000
10994,348,NhocShjnky,NaN,2010-11-20T22:42:19+0700,2010-11-20T22:54:18+0700,0.000037,0.000000,0.000000


In [ ]:
w_list=[0.3,0.3,0.4]
user['quality_score']=user.apply(lambda x: x['comment_count']*w_list[0]+ x['Reaction_Score']*w_list[1]+ x['Score']*w_list[2],axis=1)
user
                                   

,id_user,name_user,star,DateJoin,DateLastAccess,comment_count,Reaction_Score,Score,quality_score
0,179483,0406,1.0,2013-10-24T16:03:00+0700,NaN,0.878907,0.970588,1.000000,0.954849
1,468274,Clarita,1.0,2021-10-29T16:13:29+0700,2021-11-02T09:03:36+0700,0.000037,0.000000,0.000883,0.000365
2,460618,thien-ha-bet,1.0,2020-07-13T16:17:58+0700,2021-11-01T16:53:54+0700,0.000262,0.000000,0.000883,0.000432
3,468272,laptoptitan.vn,1.0,2021-10-29T10:29:04+0700,2021-10-29T10:48:30+0700,0.000037,0.000000,0.000883,0.000365
4,468270,Phúc Lucas,1.0,2021-10-29T09:42:46+0700,2021-10-31T15:20:36+0700,0.000075,0.000000,0.000883,0.000376
...,...,...,...,...,...,...,...,...,...
10991,51414,hoangpupu,NaN,2011-07-11T01:40:44+0700,2012-11-21T01:55:12+0700,0.001385,0.000000,0.000000,0.000415
10992,33680,..::Rùa::..,NaN,2011-05-26T19:21:00+0700,2013-02-27T23:20:36+0700,0.012615,0.000000,0.000000,0.003784
10993,20,^^KuTi_K_,NaN,2010-08-11T11:30:00+0700,2014-08-24T15:57:53+0700,0.014187,0.000000,0.000000,0.004256
10994,348,NhocShjnky,NaN,2010-11-20T22:42:19+0700,2010-11-20T22:54:18+0700,0.000037,0.000000,0.000000,0.000011


# 3. funtion for create profile for user

In [ ]:
def get_text(r_u, id_text):
  if(id_text==0): return pass_nan(r_u.title_post_cleaned.values[0])
  if(id_text==1): return pass_nan(r_u.content_post_cleaned.values[0])
  if(id_text==2): return pass_nan(r_u.tags_cleaned.values[0])

def pass_nan(text):
  if(text is np.nan): return ''
  return text

def Aggregate_text(id_p,title=1,content=1 ,tags=1): # Aggregate title_post, content_post, and tags into the text field.
  r_u=post[post['id_post']==id_p]
  text=''
  if(len(r_u)==0): return text
  else:
    feature=[title,content,tags]
    for i in range(3):
      if(feature[i]==1): text += get_text(r_u, i)+' '
    return text



In [ ]:
test[test['id_user']==386961]

,Unnamed: 0,id_user,id_post,label
50399,50399,386961,5510,1
50400,50400,386961,19648,0
50401,50401,386961,19961,0
50402,50402,386961,11255,0
50403,50403,386961,18353,0
...,...,...,...,...
50495,50495,386961,20632,0
50496,50496,386961,17499,0
50497,50497,386961,20530,0
50498,50498,386961,1921,0


In [ ]:
def build_profile(id_user,id_post_pass,title=1,content=1 ,tags=1):
  #lấy id_post người dùng đã đăng trên diễn đàn
  post_u_posted=np.unique(post[post['id_author_post']==int(id_user)].id_post.values)
  #lấy id_post người dùng đã comment
  post_u_commented=np.unique(comment[comment['id_author_cmt']==int(id_user)].id_post)
  #tổng hợp tất cả id_post
  post_u_interacted=list(np.unique(np.hstack((post_u_posted,post_u_commented))))
  # bỏ đi id_post trong tập 
  post_u_interacted.remove(id_post_pass)
  #build
  profile = [Aggregate_text(p,title=title,content=content ,tags=tags) for p in post_u_interacted]
  return profile

In [ ]:
def build_profile_update(id_user,id_post_pass,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1):
  #lấy id_post người dùng đã đăng trên diễn đàn
  if(p_posted==1):
    post_u_posted=np.unique(post[post['id_author_post']==int(id_user)].id_post.values)
  else: post_u_posted=np.array([])
  #lấy id_post người dùng đã comment
  if(p_commented==1):
    post_u_commented=np.unique(comment[comment['id_author_cmt']==int(id_user)].id_post)
  else: post_u_commented=np.array([])
  #tổng hợp tất cả id_post
  post_u_interacted=list(np.unique(np.hstack((post_u_posted,post_u_commented))))
  # bỏ đi id_post trong tập 
  if(id_post_pass in post_u_interacted): post_u_interacted.remove(id_post_pass)
  #build
  profile = [Aggregate_text(p,title=p_title,content=p_content ,tags=p_tags) for p in post_u_interacted]
  return profile

# 4. Embbeding

## tf-idf

In [ ]:
import gensim
import gensim.corpora as corpora 
from gensim.corpora import Dictionary

In [ ]:
docs=[Aggregate_text(p,title=1,content=1 ,tags=1) for p in post.id_post]
token_docs=[doc.split() for doc in docs]

In [ ]:
dic

In [ ]:
dic = corpora.Dictionary(token_docs)
#dic.save_as_text('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic.txt')

In [ ]:
corpus=[dic.doc2bow(r) for r in token_docs]
modeltfidf = gensim.models.TfidfModel(corpus)
#modeltfidf.save('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_tfidf.model')

In [ ]:
docs=[Aggregate_text(p,title=1,content=0 ,tags=1) for p in post.id_post]
token_docs=[doc.split() for doc in docs]
dic = corpora.Dictionary(token_docs)
dic.save_as_text('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic_111.txt')
corpus=[dic.doc2bow(r) for r in token_docs]
modeltfidf = gensim.models.TfidfModel(corpus,smartirs='lpc')
modeltfidf.save('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_tfidf_111.model')

In [ ]:
train=[Aggregate_text(id_p,title=1,content=0 ,tags=1).strip() for id_p in post.id_post.values]
train

In [ ]:
len(train)

16278

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tf_vectorizer = TfidfVectorizer(max_features=1500)
tf = tf_vectorizer.fit(train)

In [ ]:
m=tf.transform(train)

In [ ]:
# Materialize the sparse data
data_dense = m.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.2615882377032395 %


In [ ]:
data_dense

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
m[0].shape

(1, 1500)

In [ ]:
cosine_similarity(m[0].reshape(1,-1),m[1].reshape(1,-1))[0]

array([0.22751515])

In [ ]:
tf.vocabulary_

# 5. Similarity


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def quality_score_u_p(id_p):
  return user.loc[user.index[user['id_user']==post[post['id_post']==id_p].get('id_author_post').values[0]].tolist()[0],'quality_score']

In [ ]:
user

,id_user,name_user,star,DateJoin,DateLastAccess,comment_count,Reaction_Score,Score,quality_score
0,179483,0406,1.0,2013-10-24T16:03:00+0700,NaN,0.878907,0.970588,1.000000,0.954849
1,468274,Clarita,1.0,2021-10-29T16:13:29+0700,2021-11-02T09:03:36+0700,0.000037,0.000000,0.000883,0.000365
2,460618,thien-ha-bet,1.0,2020-07-13T16:17:58+0700,2021-11-01T16:53:54+0700,0.000262,0.000000,0.000883,0.000432
3,468272,laptoptitan.vn,1.0,2021-10-29T10:29:04+0700,2021-10-29T10:48:30+0700,0.000037,0.000000,0.000883,0.000365
4,468270,Phúc Lucas,1.0,2021-10-29T09:42:46+0700,2021-10-31T15:20:36+0700,0.000075,0.000000,0.000883,0.000376
...,...,...,...,...,...,...,...,...,...
10991,51414,hoangpupu,NaN,2011-07-11T01:40:44+0700,2012-11-21T01:55:12+0700,0.001385,0.000000,0.000000,0.000415
10992,33680,..::Rùa::..,NaN,2011-05-26T19:21:00+0700,2013-02-27T23:20:36+0700,0.012615,0.000000,0.000000,0.003784
10993,20,^^KuTi_K_,NaN,2010-08-11T11:30:00+0700,2014-08-24T15:57:53+0700,0.014187,0.000000,0.000000,0.004256
10994,348,NhocShjnky,NaN,2010-11-20T22:42:19+0700,2010-11-20T22:54:18+0700,0.000037,0.000000,0.000000,0.000011


In [ ]:
def get_sim_avg(check_sub,profile):
  return np.mean((cosine_similarity(check_sub.reshape(1,-1),profile)[0]))

In [ ]:
def convert_dic(tup):
  return dict((x, y) for x, y in tup)
  
def RS(id_u, model_embedding, dictionary,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  ground_truth=test.copy()
  ground_truth=ground_truth[ground_truth['id_user']==id_u]
  ground_truth['quality_score_author_p']=ground_truth.apply(lambda x: quality_score_u_p(x['id_post']),axis=1)

  id_post_pass=ground_truth[ground_truth['label']==1].id_post.values[0]
  #profile=build_profile(id_u,id_post_pass,title=1,content=1 ,tags=1)

  model=model_embedding
  dic=dictionary
  
  profile=build_profile_update(id_u,id_post_pass,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags)
  #profile_tfidf= [ convert_dic(model[dic.doc2bow(' '.join(p).split())]) for p in profile]
  profile_tfidf= tf.transform(profile)

  
  #profile_tfidf=model[dic.doc2bow(' '.join(profile).split())]
  #profile_tfidf=convert_dic(profile_tfidf)

  docs_test=[Aggregate_text(p,title=title,content=content ,tags=tags) for p in ground_truth.id_post.values]
  #docs_test_tfidf=[convert_dic(model[dic.doc2bow(doc.split())]) for doc in docs_test]
  docs_test_tfidf=tf.transform(docs_test)

  #for p in profile_tfidf: docs_test_tfidf.insert(0,p)
  #df_cal=pd.DataFrame(docs_test_tfidf).fillna(0)
  #df_cal_p=pd.DataFrame(profile_tfidf).fillna(0)

  #profile_cal=np.array(df_cal.iloc[:len(profile),:])
  #check_cal=np.array(df_cal.iloc[len(profile):,:])

  list_similarity=[get_sim_avg(doc_test,profile_tfidf) for doc_test in docs_test_tfidf]
  
  result=ground_truth.copy()
  result=result[['id_post','quality_score_author_p','label']]
  result['sim_avg']=list_similarity
  #result['sim_avg']=MinMaxScaler().fit_transform(np.array(result[['sim']]).reshape(-1, 1))
  #result['for_rank']=0.2*result['quality_score_author_p']+0.8*result['sim']
  result=result.sort_values(by=['sim_avg','quality_score_author_p'], ascending=False)
  return list(zip(*map(result.get, result)))
  #return list_similarity

In [ ]:
#load_model tfidf
model=gensim.models.TfidfModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_tfidf_111.model')
#load dic
dic = corpora.Dictionary.load_from_text('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic_111.txt')

In [ ]:
a=RS(466251, model, dic,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1)
a

[(905, 2.2459292532285233e-05, 0, 0.19214950437562034),
 (6891, 1.1229646266142617e-05, 0, 0.16627748452708097),
 (8786, 0.0017518248175182483, 0, 0.1519658414912919),
 (5960, 8.983717012914093e-05, 0, 0.1270920802749963),
 (9563, 0.0032341381246490733, 0, 0.12506303415460188),
 (21492, 0.003728242560359349, 0, 0.12225570708269458),
 (18906, 1.1229646266142617e-05, 0, 0.11479926488793284),
 (8526, 0.0024817518248175185, 0, 0.11162051484091925),
 (10377, 0.05852998771881442, 0, 0.11026096003962263),
 (216, 0.0022436674516837524, 1, 0.10534851479071543),
 (8580, 0.00012352610892756878, 0, 0.10431193356115029),
 (6911, 2.2459292532285233e-05, 0, 0.10128877614050644),
 (2405, 0.00029197080291970805, 0, 0.09211772466973447),
 (16691, 2.2459292532285233e-05, 0, 0.08916078668620224),
 (6499, 0.00026951151038742276, 0, 0.08856818859112872),
 (12036, 1.1229646266142617e-05, 0, 0.08538762705755583),
 (11879, 0.002582818641212802, 0, 0.08323419504976672),
 (3953, 6.737787759685569e-05, 0, 0.07872

In [ ]:
len(dic)

26223

In [ ]:
RS(466251, model, dic,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1)

,id_post,quality_score_author_p,sim,for_rank
117,7455,0.000011,1.000000,0.800002
174,11361,0.011039,0.846274,0.679227
187,8191,0.000146,0.816165,0.652961
113,3015,0.000011,0.702667,0.562136
127,2916,0.000045,0.696254,0.557012
...,...,...,...,...
166,16691,0.000022,0.000000,0.000004
112,9882,0.000011,0.000000,0.000002
123,8928,0.000011,0.000000,0.000002
138,18906,0.000011,0.000000,0.000002


In [ ]:
test[test['label']==1]

,Unnamed: 0,id_user,id_post,label
0,0,179483,16,1
101,101,466251,216,1
202,202,468305,14,1
303,303,111027,368,1
404,404,286410,1,1
...,...,...,...,...
49995,49995,267871,12276,1
50096,50096,342550,8377,1
50197,50197,402686,6090,1
50298,50298,277065,15390,1


# 6. RS and evaluation

In [ ]:
import gensim

In [ ]:
#load_model tfidf
model=gensim.models.TfidfModel.load('/content/drive/MyDrive/DS300/Model_RS_DA/CB/model_tfidf.model')
#load dic
dic = corpora.Dictionary.load_from_text('/content/drive/MyDrive/DS300/Model_RS_DA/CB/dic.txt')

In [ ]:
def get_sim_rank(p_true, RS_list):
  index=[x[0] for x in RS_list].index(p_true)
  return (index,RS_list[index][1])

In [ ]:
def evaluation(df,title=1,content=1 ,tags=1):
  top_list=[5,10,15,20,25]
  name_top= ['top_%s'%n for n in top_list]
  name_result=['result_top_%s'%n for n in top_list]

  check=df.copy()
  check['RS']=check['id_user'].apply(RS,model_embedding=model,dictionary=dic,title=title,content=content ,tags=tags)

  check['rank_sim_ptrue']=check.apply(lambda x: get_sim_rank(x['id_post'],x['RS']),axis = 1)
  for i in range(len(top_list)):
    check[name_top[i]]=check['RS'].apply(id_p_top,top_n=top_list[i])
  for i in range(len(top_list)):
    check[name_result[i]]=check.apply(lambda x: 1 if x['id_post'] in x[name_top[i]] else 0,axis = 1)

  acc=[list(check[n].values).count(1)/len(check) for n in name_result ]

  return check,acc


### e1
* tf_idf fit in post full: title=1,content=1 ,tags=1
* cosine
* build profile with p_posted + p_commented : title=1,content=1,tags=1
* get_doc_test for similarity: title=1,content=1,tags=1

In [ ]:
e1=evaluation(test[test['label']==1],title=1,content=1,tags=1)
df=e1[0]
acc=e1[1]
acc

[0.05, 0.122, 0.172, 0.24, 0.286]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(13905, 0.26953244765912077), (13075, 0.14457...","(60, 0.0)","[13905, 13075, 17312, 13442, 17555]","[13905, 13075, 17312, 13442, 17555, 7472, 1276...","[13905, 13075, 17312, 13442, 17555, 7472, 1276...","[13905, 13075, 17312, 13442, 17555, 7472, 1276...","[13905, 13075, 17312, 13442, 17555, 7472, 1276...",0,0,0,0,0
101,101,466251,216,1,"[(7455, 0.14386104621006454), (11361, 0.121745...","(57, 0.0)","[7455, 11361, 8191, 3015, 2916]","[7455, 11361, 8191, 3015, 2916, 905, 21514, 14...","[7455, 11361, 8191, 3015, 2916, 905, 21514, 14...","[7455, 11361, 8191, 3015, 2916, 905, 21514, 14...","[7455, 11361, 8191, 3015, 2916, 905, 21514, 14...",0,0,0,0,0
202,202,468305,14,1,"[(9941, 0.09438189383723061), (5296, 0.0822314...","(33, 0.0)","[9941, 5296, 12065, 5021, 15173]","[9941, 5296, 12065, 5021, 15173, 440, 12683, 1...","[9941, 5296, 12065, 5021, 15173, 440, 12683, 1...","[9941, 5296, 12065, 5021, 15173, 440, 12683, 1...","[9941, 5296, 12065, 5021, 15173, 440, 12683, 1...",0,0,0,0,0
303,303,111027,368,1,"[(3268, 0.12264318933474248), (14711, 0.110773...","(56, 0.0)","[3268, 14711, 19354, 1482, 14278]","[3268, 14711, 19354, 1482, 14278, 10622, 3859,...","[3268, 14711, 19354, 1482, 14278, 10622, 3859,...","[3268, 14711, 19354, 1482, 14278, 10622, 3859,...","[3268, 14711, 19354, 1482, 14278, 10622, 3859,...",0,0,0,0,0
404,404,286410,1,1,"[(18033, 0.1629977881588284), (4534, 0.1423370...","(57, 0.0)","[18033, 4534, 12946, 17182, 19293]","[18033, 4534, 12946, 17182, 19293, 14617, 1975...","[18033, 4534, 12946, 17182, 19293, 14617, 1975...","[18033, 4534, 12946, 17182, 19293, 14617, 1975...","[18033, 4534, 12946, 17182, 19293, 14617, 1975...",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(8719, 0.10053034694652906), (4076, 0.0709684...","(29, 0.0)","[8719, 4076, 18298, 15343, 9572]","[8719, 4076, 18298, 15343, 9572, 18203, 16386,...","[8719, 4076, 18298, 15343, 9572, 18203, 16386,...","[8719, 4076, 18298, 15343, 9572, 18203, 16386,...","[8719, 4076, 18298, 15343, 9572, 18203, 16386,...",0,0,0,0,0
50096,50096,342550,8377,1,"[(9553, 0.19951994498762968), (13417, 0.107461...","(34, 0.0)","[9553, 13417, 13946, 6377, 19942]","[9553, 13417, 13946, 6377, 19942, 20713, 14076...","[9553, 13417, 13946, 6377, 19942, 20713, 14076...","[9553, 13417, 13946, 6377, 19942, 20713, 14076...","[9553, 13417, 13946, 6377, 19942, 20713, 14076...",0,0,0,0,0
50197,50197,402686,6090,1,"[(16498, 0.12753556317837567), (14980, 0.10303...","(33, 0.0)","[16498, 14980, 10294, 10124, 886]","[16498, 14980, 10294, 10124, 886, 20921, 1535,...","[16498, 14980, 10294, 10124, 886, 20921, 1535,...","[16498, 14980, 10294, 10124, 886, 20921, 1535,...","[16498, 14980, 10294, 10124, 886, 20921, 1535,...",0,0,0,0,0
50298,50298,277065,15390,1,"[(5905, 0.2260057185648619), (17182, 0.0727427...","(55, 0.0)","[5905, 17182, 779, 7527, 19532]","[5905, 17182, 779, 7527, 19532, 5679, 13006, 1...","[5905, 17182, 779, 7527, 19532, 5679, 13006, 1...","[5905, 17182, 779, 7527, 19532, 5679, 13006, 1...","[5905, 17182, 779, 7527, 19532, 5679, 13006, 1...",0,0,0,0,0


In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e2_tfidf_cosine_p_full_111_doc_111_0.05_0.122_0.172_0.24_0.286.csv')

###e2
* tf_idf fit in post full: title=1,content=1 ,tags=1

* cosine

* build profile with p_posted + p_commented : title=1,content=1,tags=1

* get_doc_test for similarity: title=1,content=0,tags=1


In [ ]:
e2=evaluation(test[test['label']==1],title=1,content=0,tags=1)
df=e2[0]
acc=e2[1]
acc

KeyboardInterrupt: ignored

In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e2_tfidf_cosine_p_full_111_doc_101_0.546_0.656_0.784_0.814_0.814.csv')

### e3
* tf_idf fit in post full: title=1,content=1 ,tags=1
* cosine
* build profile with p_posted + p_commented : title=1,content=1,tags=1
* get_doc_test for similarity: title=0,content=1,tags=1

In [ ]:
e3=evaluation(test[test['label']==1],title=0,content=1,tags=1)
df=e3[0]
acc=e3[1]
acc

[0.044, 0.11, 0.168, 0.238, 0.29]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(13905, 0.3330026771534978), (13075, 0.168452...","(59, 0.0)","[13905, 13075, 17312, 12767, 7472]","[13905, 13075, 17312, 12767, 7472, 5679, 17555...","[13905, 13075, 17312, 12767, 7472, 5679, 17555...","[13905, 13075, 17312, 12767, 7472, 5679, 17555...","[13905, 13075, 17312, 12767, 7472, 5679, 17555...",0,0,0,0,0
101,101,466251,216,1,"[(7455, 0.15428885023397113), (3015, 0.1444278...","(56, 0.0)","[7455, 3015, 11361, 8191, 2916]","[7455, 3015, 11361, 8191, 2916, 905, 21514, 12...","[7455, 3015, 11361, 8191, 2916, 905, 21514, 12...","[7455, 3015, 11361, 8191, 2916, 905, 21514, 12...","[7455, 3015, 11361, 8191, 2916, 905, 21514, 12...",0,0,0,0,0
202,202,468305,14,1,"[(5296, 0.11879983981318287), (9941, 0.0959560...","(33, 0.0)","[5296, 9941, 12065, 5021, 15173]","[5296, 9941, 12065, 5021, 15173, 440, 12683, 1...","[5296, 9941, 12065, 5021, 15173, 440, 12683, 1...","[5296, 9941, 12065, 5021, 15173, 440, 12683, 1...","[5296, 9941, 12065, 5021, 15173, 440, 12683, 1...",0,0,0,0,0
303,303,111027,368,1,"[(3268, 0.14086574365677806), (19354, 0.104581...","(56, 0.0)","[3268, 19354, 1482, 14278, 12329]","[3268, 19354, 1482, 14278, 12329, 3859, 12065,...","[3268, 19354, 1482, 14278, 12329, 3859, 12065,...","[3268, 19354, 1482, 14278, 12329, 3859, 12065,...","[3268, 19354, 1482, 14278, 12329, 3859, 12065,...",0,0,0,0,0
404,404,286410,1,1,"[(4534, 0.2026267748710266), (18033, 0.1844723...","(57, 0.0)","[4534, 18033, 12946, 17182, 19293]","[4534, 18033, 12946, 17182, 19293, 3026, 18717...","[4534, 18033, 12946, 17182, 19293, 3026, 18717...","[4534, 18033, 12946, 17182, 19293, 3026, 18717...","[4534, 18033, 12946, 17182, 19293, 3026, 18717...",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(8719, 0.10942468173027914), (4076, 0.0783770...","(29, 0.0)","[8719, 4076, 18203, 18298, 15343]","[8719, 4076, 18203, 18298, 15343, 9572, 9272, ...","[8719, 4076, 18203, 18298, 15343, 9572, 9272, ...","[8719, 4076, 18203, 18298, 15343, 9572, 9272, ...","[8719, 4076, 18203, 18298, 15343, 9572, 9272, ...",0,0,0,0,0
50096,50096,342550,8377,1,"[(9553, 0.2875764352236227), (13417, 0.1333137...","(23, 0.0)","[9553, 13417, 6377, 19942, 13946]","[9553, 13417, 6377, 19942, 13946, 14076, 12495...","[9553, 13417, 6377, 19942, 13946, 14076, 12495...","[9553, 13417, 6377, 19942, 13946, 14076, 12495...","[9553, 13417, 6377, 19942, 13946, 14076, 12495...",0,0,0,0,1
50197,50197,402686,6090,1,"[(16498, 0.13150811943088214), (10294, 0.07518...","(33, 0.0)","[16498, 10294, 10124, 886, 14991]","[16498, 10294, 10124, 886, 14991, 20921, 1535,...","[16498, 10294, 10124, 886, 14991, 20921, 1535,...","[16498, 10294, 10124, 886, 14991, 20921, 1535,...","[16498, 10294, 10124, 886, 14991, 20921, 1535,...",0,0,0,0,0
50298,50298,277065,15390,1,"[(5905, 0.2962370555946357), (779, 0.077167787...","(25, 0.0)","[5905, 779, 17182, 7527, 5679]","[5905, 779, 17182, 7527, 5679, 19532, 14123, 1...","[5905, 779, 17182, 7527, 5679, 19532, 14123, 1...","[5905, 779, 17182, 7527, 5679, 19532, 14123, 1...","[5905, 779, 17182, 7527, 5679, 19532, 14123, 1...",0,0,0,0,0


In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e3_tfidf_cosine_p_full_111_doc_011_0.044_0.11_0.168_0.238_0.29.csv')

In [ ]:
def id_p_top(list_rs,top_n):
  return list_rs[:top_n]


In [ ]:
def evaluation_update(df,p_posted=1,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=1 ,tags=1):
  top_list=[5,10,15,20,25]
  name_top= ['top_%s'%n for n in top_list]
  name_result=['result_top_%s'%n for n in top_list]

  check=df.copy()
  check['RS']=check['id_user'].apply(RS,model_embedding=model,dictionary=dic,p_posted=p_posted,p_commented=p_commented,p_title=p_title,p_content=p_content ,p_tags=p_tags,title=title,content=content ,tags=tags)

  check['rank_sim_ptrue']=check.apply(lambda x: get_sim_rank(x['id_post'],x['RS']),axis = 1)
  for i in range(len(top_list)):
    check[name_top[i]]=check['RS'].apply(id_p_top,top_n=top_list[i])
  for i in range(len(top_list)):
    check[name_result[i]]=check.apply(lambda x: 1 if x['id_post'] in x[name_top[i]] else 0,axis = 1)

  acc=[list(check[n].values).count(1)/len(check) for n in name_result ]

  return check,acc


In [ ]:
e_c=evaluation_update(test[test['label']==1],p_posted=1,p_commented=1,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
df=e_c[0]
acc=e_c[1]
acc

[0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(20796, 4.4918585064570466e-05, 0, 0.06885495...","(40, 0.00037581618299164924)","[(20796, 4.4918585064570466e-05, 0, 0.06885495...","[(20796, 4.4918585064570466e-05, 0, 0.06885495...","[(20796, 4.4918585064570466e-05, 0, 0.06885495...","[(20796, 4.4918585064570466e-05, 0, 0.06885495...","[(20796, 4.4918585064570466e-05, 0, 0.06885495...",0,0,0,0,0
101,101,466251,216,1,"[(216, 0.0022436674516837524, 1, 0.10523071188...","(0, 0.0022436674516837524)","[(216, 0.0022436674516837524, 1, 0.10523071188...","[(216, 0.0022436674516837524, 1, 0.10523071188...","[(216, 0.0022436674516837524, 1, 0.10523071188...","[(216, 0.0022436674516837524, 1, 0.10523071188...","[(216, 0.0022436674516837524, 1, 0.10523071188...",0,0,0,0,0
202,202,468305,14,1,"[(3311, 0.00015721504772599665, 0, 0.100011826...","(82, 0.0004319644143223623)","[(3311, 0.00015721504772599665, 0, 0.100011826...","[(3311, 0.00015721504772599665, 0, 0.100011826...","[(3311, 0.00015721504772599665, 0, 0.100011826...","[(3311, 0.00015721504772599665, 0, 0.100011826...","[(3311, 0.00015721504772599665, 0, 0.100011826...",0,0,0,0,0
303,303,111027,368,1,"[(10521, 0.00029197080291970805, 0, 0.07999675...","(23, 0.0003645865367255066)","[(10521, 0.00029197080291970805, 0, 0.07999675...","[(10521, 0.00029197080291970805, 0, 0.07999675...","[(10521, 0.00029197080291970805, 0, 0.07999675...","[(10521, 0.00029197080291970805, 0, 0.07999675...","[(10521, 0.00029197080291970805, 0, 0.07999675...",0,0,0,0,0
404,404,286410,1,1,"[(21413, 0.00012352610892756878, 0, 0.09398293...","(75, 0.9548485649172639)","[(21413, 0.00012352610892756878, 0, 0.09398293...","[(21413, 0.00012352610892756878, 0, 0.09398293...","[(21413, 0.00012352610892756878, 0, 0.09398293...","[(21413, 0.00012352610892756878, 0, 0.09398293...","[(21413, 0.00012352610892756878, 0, 0.09398293...",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(10213, 0.0007860752386299831, 0, 0.074568166...","(67, 0.011038742279618192)","[(10213, 0.0007860752386299831, 0, 0.074568166...","[(10213, 0.0007860752386299831, 0, 0.074568166...","[(10213, 0.0007860752386299831, 0, 0.074568166...","[(10213, 0.0007860752386299831, 0, 0.074568166...","[(10213, 0.0007860752386299831, 0, 0.074568166...",0,0,0,0,0
50096,50096,342550,8377,1,"[(2569, 2.2459292532285233e-05, 0, 0.141566044...","(4, 4.4918585064570466e-05)","[(2569, 2.2459292532285233e-05, 0, 0.141566044...","[(2569, 2.2459292532285233e-05, 0, 0.141566044...","[(2569, 2.2459292532285233e-05, 0, 0.141566044...","[(2569, 2.2459292532285233e-05, 0, 0.141566044...","[(2569, 2.2459292532285233e-05, 0, 0.141566044...",0,0,0,0,0
50197,50197,402686,6090,1,"[(19831, 3.3688938798427845e-05, 0, 0.06923705...","(43, 0.00014598540145985403)","[(19831, 3.3688938798427845e-05, 0, 0.06923705...","[(19831, 3.3688938798427845e-05, 0, 0.06923705...","[(19831, 3.3688938798427845e-05, 0, 0.06923705...","[(19831, 3.3688938798427845e-05, 0, 0.06923705...","[(19831, 3.3688938798427845e-05, 0, 0.06923705...",0,0,0,0,0
50298,50298,277065,15390,1,"[(19050, 0.0005502526670409882, 0, 0.087107570...","(13, 0.00046041549691184727)","[(19050, 0.0005502526670409882, 0, 0.087107570...","[(19050, 0.0005502526670409882, 0, 0.087107570...","[(19050, 0.0005502526670409882, 0, 0.087107570...","[(19050, 0.0005502526670409882, 0, 0.087107570...","[(19050, 0.0005502526670409882, 0, 0.087107570...",0,0,0,0,0


### e4
* tf_idf fit in post full: title=1,content=1 ,tags=1
* cosine
* build profile with p_posted=1 + p_commented=0 : title=1,content=1,tags=1
* get_doc_test for similarity: title=1,content=0,tags=1

In [ ]:
e4=evaluation_update(test[test['label']==1],p_posted=1,p_commented=0,p_title=1,p_content=1 ,p_tags=1,title=1,content=0 ,tags=1)
df=e4[0]
acc=e4[1]
acc

[0.7, 0.77, 0.862, 0.88, 0.88]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(7472, 0.12255420563753625), (13442, 0.116752...","(7, 0.0)","[7472, 13442, 5803, 15059, 13946]","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...",0,1,1,1,1
101,101,466251,216,1,"[(216, 0.0), (6911, 0.0), (20080, 0.0), (11361...","(0, 0.0)","[216, 6911, 20080, 11361, 16357]","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...",1,1,1,1,1
202,202,468305,14,1,"[(14, 0.0), (14960, 0.0), (4786, 0.0), (5170, ...","(0, 0.0)","[14, 14960, 4786, 5170, 3773]","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...",1,1,1,1,1
303,303,111027,368,1,"[(1482, 0.10656503457006297), (10622, 0.098322...","(5, 0.0)","[1482, 10622, 14711, 5083, 19878]","[1482, 10622, 14711, 5083, 19878, 368, 16176, ...","[1482, 10622, 14711, 5083, 19878, 368, 16176, ...","[1482, 10622, 14711, 5083, 19878, 368, 16176, ...","[1482, 10622, 14711, 5083, 19878, 368, 16176, ...",0,1,1,1,1
404,404,286410,1,1,"[(18033, 0.07706407861204018), (13936, 0.02363...","(6, 0.0)","[18033, 13936, 17182, 924, 19890]","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...",0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(4076, 0.08585278635227071), (9572, 0.0241319...","(5, 0.0)","[4076, 9572, 16386, 11348, 15343]","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...",0,1,1,1,1
50096,50096,342550,8377,1,"[(13946, 0.0386924146440623), (19942, 0.022003...","(4, 0.0)","[13946, 19942, 18292, 20713, 8377]","[13946, 19942, 18292, 20713, 8377, 12495, 5267...","[13946, 19942, 18292, 20713, 8377, 12495, 5267...","[13946, 19942, 18292, 20713, 8377, 12495, 5267...","[13946, 19942, 18292, 20713, 8377, 12495, 5267...",1,1,1,1,1
50197,50197,402686,6090,1,"[(6090, 0.0), (16867, 0.0), (10151, 0.0), (166...","(0, 0.0)","[6090, 16867, 10151, 16654, 15386]","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...",1,1,1,1,1
50298,50298,277065,15390,1,"[(14750, 0.019611056998563397), (17182, 0.0177...","(2, 0.0)","[14750, 17182, 15390, 8395, 9059]","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...",1,1,1,1,1


In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e4_tfidf_cosine_p_10_111_doc_101_0.7_0.77_0.862_0.88_0.88.csv')

### e5
* tf_idf fit in post full: title=1,content=1 ,tags=1
* cosine
* build profile with p_posted=0 + p_commented=1 : title=1,content=1,tags=1
* get_doc_test for similarity: title=1,content=0,tags=1

In [ ]:
e5=evaluation_update(test[test['label']==1],p_posted=0,p_commented=1,p_title=1,p_content=1 ,p_tags=1,title=1,content=0 ,tags=1)
df=e5[0]
acc=e5[1]
acc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[0.546, 0.654, 0.786, 0.816, 0.816]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(13442, 0.12195594101810303), (7472, 0.114986...","(7, 0.0)","[13442, 7472, 15059, 5803, 13946]","[13442, 7472, 15059, 5803, 13946, 13905, 1384,...","[13442, 7472, 15059, 5803, 13946, 13905, 1384,...","[13442, 7472, 15059, 5803, 13946, 13905, 1384,...","[13442, 7472, 15059, 5803, 13946, 13905, 1384,...",0,1,1,1,1
101,101,466251,216,1,"[(12312, 0.031062317829118516), (19108, 0.0282...","(3, 0.0)","[12312, 19108, 6976, 216, 16691]","[12312, 19108, 6976, 216, 16691, 11361, 16357,...","[12312, 19108, 6976, 216, 16691, 11361, 16357,...","[12312, 19108, 6976, 216, 16691, 11361, 16357,...","[12312, 19108, 6976, 216, 16691, 11361, 16357,...",1,1,1,1,1
202,202,468305,14,1,"[(14, 0.0), (14960, 0.0), (4786, 0.0), (5170, ...","(0, 0.0)","[14, 14960, 4786, 5170, 3773]","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...",1,1,1,1,1
303,303,111027,368,1,"[(14711, 0.12951444181852387), (1482, 0.116814...","(6, 0.0)","[14711, 1482, 5083, 10622, 19878]","[14711, 1482, 5083, 10622, 19878, 14249, 368, ...","[14711, 1482, 5083, 10622, 19878, 14249, 368, ...","[14711, 1482, 5083, 10622, 19878, 14249, 368, ...","[14711, 1482, 5083, 10622, 19878, 14249, 368, ...",0,1,1,1,1
404,404,286410,1,1,"[(18033, 0.08511714004727138), (17182, 0.02654...","(6, 0.0)","[18033, 17182, 13936, 924, 19890]","[18033, 17182, 13936, 924, 19890, 15944, 1, 18...","[18033, 17182, 13936, 924, 19890, 15944, 1, 18...","[18033, 17182, 13936, 924, 19890, 15944, 1, 18...","[18033, 17182, 13936, 924, 19890, 15944, 1, 18...",0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(4076, 0.10117462857865875), (9572, 0.0290314...","(5, 0.0)","[4076, 9572, 16386, 11348, 15343]","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...",0,1,1,1,1
50096,50096,342550,8377,1,"[(13946, 0.03215846406760018), (19942, 0.03086...","(4, 0.0)","[13946, 19942, 20713, 18292, 8377]","[13946, 19942, 20713, 18292, 8377, 12495, 5267...","[13946, 19942, 20713, 18292, 8377, 12495, 5267...","[13946, 19942, 20713, 18292, 8377, 12495, 5267...","[13946, 19942, 20713, 18292, 8377, 12495, 5267...",1,1,1,1,1
50197,50197,402686,6090,1,"[(14980, 0.103031696044272), (13124, 0.0076458...","(3, 0.0)","[14980, 13124, 20105, 6090, 13274]","[14980, 13124, 20105, 6090, 13274, 16654, 1538...","[14980, 13124, 20105, 6090, 13274, 16654, 1538...","[14980, 13124, 20105, 6090, 13274, 16654, 1538...","[14980, 13124, 20105, 6090, 13274, 16654, 1538...",1,1,1,1,1
50298,50298,277065,15390,1,"[(17182, 0.027927107452307046), (14750, 0.0247...","(2, 0.0)","[17182, 14750, 15390, 8395, 9059]","[17182, 14750, 15390, 8395, 9059, 20498, 17130...","[17182, 14750, 15390, 8395, 9059, 20498, 17130...","[17182, 14750, 15390, 8395, 9059, 20498, 17130...","[17182, 14750, 15390, 8395, 9059, 20498, 17130...",1,1,1,1,1


In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e5_tfidf_cosine_p_01_111_doc_101_0.546_0.654_0.786_0.816_0.816.csv')

### e6
* tf_idf fit in post full: title=1,content=1 ,tags=1
* cosine
* build profile with p_posted=1 + p_commented=0 : title=1,content=0,tags=1
* get_doc_test for similarity: title=1,content=0,tags=1

In [ ]:
e6=evaluation_update(test[test['label']==1],p_posted=1,p_commented=0,p_title=1,p_content=0 ,p_tags=1,title=1,content=0 ,tags=1)
df=e6[0]
acc=e6[1]
acc

[0.774, 0.818, 0.89, 0.9, 0.9]

In [ ]:
df

,Unnamed: 0,id_user,id_post,label,RS,rank_sim_ptrue,top_5,top_10,top_15,top_20,top_25,result_top_5,result_top_10,result_top_15,result_top_20,result_top_25
0,0,179483,16,1,"[(7472, 0.14937626265154966), (13442, 0.115563...","(7, 0.0)","[7472, 13442, 5803, 15059, 13946]","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...","[7472, 13442, 5803, 15059, 13946, 13905, 1384,...",0,1,1,1,1
101,101,466251,216,1,"[(216, 0.0), (6911, 0.0), (20080, 0.0), (11361...","(0, 0.0)","[216, 6911, 20080, 11361, 16357]","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...","[216, 6911, 20080, 11361, 16357, 8208, 14500, ...",1,1,1,1,1
202,202,468305,14,1,"[(14, 0.0), (14960, 0.0), (4786, 0.0), (5170, ...","(0, 0.0)","[14, 14960, 4786, 5170, 3773]","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...","[14, 14960, 4786, 5170, 3773, 2508, 7250, 1664...",1,1,1,1,1
303,303,111027,368,1,"[(10622, 0.15904749946335162), (1482, 0.143904...","(2, 0.0)","[10622, 1482, 368, 19903, 3065]","[10622, 1482, 368, 19903, 3065, 16176, 16790, ...","[10622, 1482, 368, 19903, 3065, 16176, 16790, ...","[10622, 1482, 368, 19903, 3065, 16176, 16790, ...","[10622, 1482, 368, 19903, 3065, 16176, 16790, ...",1,1,1,1,1
404,404,286410,1,1,"[(18033, 0.10011209762672005), (13936, 0.03397...","(6, 0.0)","[18033, 13936, 17182, 924, 19890]","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...","[18033, 13936, 17182, 924, 19890, 15944, 1, 18...",0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,267871,12276,1,"[(4076, 0.10098991929923208), (9572, 0.0345438...","(5, 0.0)","[4076, 9572, 16386, 11348, 15343]","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...","[4076, 9572, 16386, 11348, 15343, 12276, 13170...",0,1,1,1,1
50096,50096,342550,8377,1,"[(13946, 0.10899852593408353), (8377, 0.0), (1...","(1, 0.0)","[13946, 8377, 18021, 5267, 9553]","[13946, 8377, 18021, 5267, 9553, 12996, 11139,...","[13946, 8377, 18021, 5267, 9553, 12996, 11139,...","[13946, 8377, 18021, 5267, 9553, 12996, 11139,...","[13946, 8377, 18021, 5267, 9553, 12996, 11139,...",1,1,1,1,1
50197,50197,402686,6090,1,"[(6090, 0.0), (16867, 0.0), (10151, 0.0), (166...","(0, 0.0)","[6090, 16867, 10151, 16654, 15386]","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...","[6090, 16867, 10151, 16654, 15386, 16980, 1317...",1,1,1,1,1
50298,50298,277065,15390,1,"[(14750, 0.023274676137189088), (17182, 0.0093...","(2, 0.0)","[14750, 17182, 15390, 8395, 9059]","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...","[14750, 17182, 15390, 8395, 9059, 20498, 17130...",1,1,1,1,1


In [ ]:
df.iloc[['101']]['RS'].values

array([list([(13953, 0.07912237157064442), (5608, 0.017403514708492447), (19719, 0.0), (5620, 0.0), (6789, 0.0), (11983, 0.0), (12275, 0.0), (13643, 0.0), (14369, 0.0), (9731, 0.0), (2471, 0.0), (17160, 0.0), (3833, 0.0), (11932, 0.0), (4093, 0.0), (8289, 0.0), (17905, 0.0), (18975, 0.0), (19136, 0.0), (21129, 0.0), (20241, 0.0), (18938, 0.0), (21517, 0.0), (9737, 0.0), (2022, 0.0), (6966, 0.0), (4011, 0.0), (9910, 0.0), (15791, 0.0), (10384, 0.0), (7282, 0.0), (4304, 0.0), (15261, 0.0), (20429, 0.0), (20985, 0.0), (17574, 0.0), (752, 0.0), (15406, 0.0), (6825, 0.0), (10420, 0.0), (12251, 0.0), (6328, 0.0), (6049, 0.0), (15539, 0.0), (824, 0.0), (3716, 0.0), (8322, 0.0), (14002, 0.0), (12910, 0.0), (21578, 0.0), (5500, 0.0), (11592, 0.0), (12706, 0.0), (7110, 0.0), (15549, 0.0), (308, 0.0), (12680, 0.0), (16276, 0.0), (20354, 0.0), (19000, 0.0), (9697, 0.0), (12169, 0.0), (6786, 0.0), (5492, 0.0), (12655, 0.0), (18892, 0.0), (532, 0.0), (16192, 0.0), (20090, 0.0), (403, 0.0), (2767, 0.

In [ ]:
df.to_csv('/content/drive/MyDrive/DS300/Model_RS_DA/CB/Result/tf_idf/e6_tfidf_cosine_p_10_101_doc_101_0.774_0.818_0.89_0.9_0.9.csv')